In [1]:
pip install torch torchvision efficientnet_pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/159.8 MB 1.7 MB/s eta 0:01:32
   ---------------------------------------- 0.2/159.8 MB 2.2 MB/s eta 0:01:12
   ---------------------------------------- 0.3/159.8 MB 2.5 MB/s eta 0:01:03
   ---------------------------------------- 0.5/159.8 MB 3.1 MB/s eta 0:00:52
   ---------------------------------------- 1.0/159.8 MB 4.8 MB/s eta 0:00:33
   ---------------------------------------- 1.0/159.8 MB 5.1 MB/s eta 0:00:32
   ---------------------------------------- 1.5/159.8 MB 5.3 MB/s eta 0:00:30
   ---------------------------------------- 2.0/159.8 MB

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "C:\\Users\\hp\\OneDrive\\5th semester\\archive (16)"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
model = EfficientNet.from_pretrained('efficientnet-b0')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\hp/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:03<00:00, 6.42MB/s]

Loaded pretrained weights for efficientnet-b0


In [6]:
import copy

def train_model(model, criterion, optimizer, scheduler, num_epochs=25, patience=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                patience_counter = 0
            elif phase == 'test':
                patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    print(f'Best val Acc: {best_acc:4f}')
    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)


Epoch 0/24
----------
train Loss: 2.3860 Acc: 0.2922
test Loss: 2.2996 Acc: 0.3463
Epoch 1/24
----------
train Loss: 2.0882 Acc: 0.3802
test Loss: 1.9529 Acc: 0.4328
Epoch 2/24
----------
train Loss: 1.9328 Acc: 0.4209
test Loss: 1.8649 Acc: 0.4403
Epoch 3/24
----------
train Loss: 1.8150 Acc: 0.4522
test Loss: 1.7259 Acc: 0.4915
Epoch 4/24
----------
train Loss: 1.7235 Acc: 0.4762
test Loss: 1.7667 Acc: 0.4803
Epoch 5/24
----------
train Loss: 1.6271 Acc: 0.5072
test Loss: 1.5993 Acc: 0.5207
Epoch 6/24
----------
train Loss: 1.5499 Acc: 0.5290
test Loss: 1.7090 Acc: 0.5067
Epoch 7/24
----------
train Loss: 1.2683 Acc: 0.6096
test Loss: 1.3061 Acc: 0.6192
Epoch 8/24
----------
train Loss: 1.1459 Acc: 0.6473
test Loss: 1.2679 Acc: 0.6264
Epoch 9/24
----------
train Loss: 1.0833 Acc: 0.6630
test Loss: 1.2505 Acc: 0.6359
Epoch 10/24
----------
train Loss: 1.0283 Acc: 0.6771
test Loss: 1.2490 Acc: 0.6429
Epoch 11/24
----------
train Loss: 0.9964 Acc: 0.6873
test Loss: 1.2289 Acc: 0.6444
Ep

EFFICIENTNET - B3

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "C:\\Users\\hp\\OneDrive\\5th semester\\archive (16)"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [8]:
model = EfficientNet.from_pretrained('efficientnet-b3')
num_ftrs = model._fc.in_features
model._fc = torch.nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to C:\Users\hp/.cache\torch\hub\checkpoints\efficientnet-b3-5fb5a3c3.pth
100%|██████████| 47.1M/47.1M [00:08<00:00, 5.88MB/s]


Loaded pretrained weights for efficientnet-b3


C:\Users\hp\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
import copy

def train_model(model, criterion, optimizer, scheduler, num_epochs=25, patience=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'test':
                scheduler.step(epoch_loss)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                else:
                    patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 2.5610 Acc: 0.2434
test Loss: 2.8231 Acc: 0.2501
Epoch 1/24
----------
train Loss: 2.3231 Acc: 0.3102
test Loss: 2.2918 Acc: 0.3526
Epoch 2/24
----------
train Loss: 2.1980 Acc: 0.3487
test Loss: 2.1199 Acc: 0.3728
Epoch 3/24
----------
train Loss: 2.1047 Acc: 0.3692
test Loss: 1.9264 Acc: 0.4315
Epoch 4/24
----------
